![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Abstracting Real World Data from Oncology Notes: Data Analysis

In the previous notebook (`./15-entity-extraction`) we used SparkNLP's pipelines to extract hightly specialized oncological entities from unstructured notes and stored the resulting tabular data in our delta lake.

In this notebook we analyze these data to answer questions such as:
What are the most common cancer subtypes? What are the most common symptoms and how are these symptoms associated with each cancer subtype? which indications have the highest risk factor? etc.

#0. Initial configurations

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 4.2.4
sparknlp_jsl.version : 4.2.4
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

In [0]:
delta_path='/FileStore/HLS/nlp/delta/jsl/'

let's take a look at the raw text dataset

In [0]:
df=spark.read.load(f'{delta_path}/bronze/mt-oc-notes')
display(df)

path text note_id patient_id dbfs:/FileStore/HLS/nlp/data/mt_oncology_10/mt_note_01.txt 
Medical Specialty:Hematology - Oncology
Sample Name: BRCA-2 mutation
Description: Discharge summary of a patient with a BRCA-2 mutation.
(Medical Transcription Sample Report)
DISCHARGE DIAGNOSES: BRCA-2 mutation.
HISTORY OF PRESENT ILLNESS: The patient is a 59-year-old with a BRCA-2 mutation. Her sister died of breast cancer at age 32 and her daughter had breast cancer at age 27.
PHYSICAL EXAMINATION: The chest was clear. The abdomen was nontender. Pelvic examination shows no masses. No heart murmur.
HOSPITAL COURSE: The patient underwent surgery on the day of admission. In the postoperative course she was afebrile and unremarkable. The patient regained bowel function and was discharged on the morning of the fourth postoperative day.
OPERATIONS AND PROCEDURES: Total abdominal hysterectomy/bilateral salpingo-oophorectomy with resection of ovarian fossa peritoneum en bloc on July 25, 2006.
PATHOLOGY: A 105-gram uterus without dysplasia or cancer.
CONDITION ON DISCHARGE: Stable.
PLAN: The patient will remain at rest initially with progressive ambulation after. She will avoid lifting, driving or intercourse. She will call me if any fevers, drainage, bleeding, or pain. Follow up in my office in four weeks. Family history, social history, psychosocial needs per the social worker.
DISCHARGE MEDICATIONS: Percocet 5 #40 one every 3 hours p.r.n. pain.
 null null dbfs:/FileStore/HLS/nlp/data/mt_oncology_10/mt_note_02.txt Medical Specialty:Hematology - Oncology
Sample Name: Mullerian Adenosarcoma
Description: Discharge summary of a patient presenting with a large mass aborted through the cervix.
(Medical Transcription Sample Report)
PRINCIPAL DIAGNOSIS: Mullerian adenosarcoma.
HISTORY OF PRESENT ILLNESS: The patient is a 56-year-old presenting with a large mass aborted through the cervix.
PHYSICAL EXAM:CHEST: Clear. There is no heart murmur. ABDOMEN: Nontender.
PELVIC: There is a large mass in the vagina.
HOSPITAL COURSE: The patient went to surgery on the day of admission. The postoperative course was marked by fever and ileus. The patient regained bowel function. She was discharged on the morning of the seventh postoperative day.
OPERATIONS: July 25, 2006: Total abdominal hysterectomy, bilateral salpingo-oophorectomy.
DISCHARGE CONDITION: Stable.
PLAN: The patient will remain at rest initially with progressive ambulation thereafter. She will avoid lifting, driving, stairs, or intercourse. She will call me for fevers, drainage, bleeding, or pain. Family history, social history, and psychosocial needs per the social worker. The patient will follow up in my office in one week.
PATHOLOGY: Mullerian adenosarcoma.
MEDICATIONS: Percocet 5, #40, one q.3 h. p.r.n. pain.
 null null dbfs:/FileStore/HLS/nlp/data/mt_oncology_10/mt_note_03.txt Medical Specialty:Hematology - Oncology
Sample Name: Leiomyosarcoma
Description: Discharge summary of patient with leiomyosarcoma and history of pulmonary embolism, subdural hematoma, pancytopenia, and pneumonia.
(Medical Transcription Sample Report)
ADMITTING DIAGNOSES:1. Leiomyosarcoma.2. History of pulmonary embolism.3. History of subdural hematoma.4. Pancytopenia.5. History of pneumonia.
PROCEDURES DURING HOSPITALIZATION:1. Cycle six of CIVI-CAD (Cytoxan, Adriamycin, and DTIC) from 07/22/2008 to 07/29/2008.2. CTA, chest PE study showing no evidence for pulmonary embolism.
3. Head CT showing no evidence of acute intracranial abnormalities.4. Sinus CT, normal mini-CT of the paranasal sinuses.
HISTORY OF PRESENT ILLNESS: Ms. ABC is a pleasant 66-year-old Caucasian female who first palpated a mass in the left posterior arm in spring of 2007. The mass increased in size and she was seen by her primary care physician and referred to orthopedic surgeon. MRI showed inflammation and was thought to be secondary to rheumatoid arthritis. The mass increased in size. She eventually underwent a partial resection found to have pathologi

# 1. ICD-10 Codes and HCC Status

Now we load the `icd10` delta tables

In [0]:
icd10_hcc_df=spark.read.load(f'{delta_path}/silver/icd10-hcc-df')
icd10_hcc_df.createOrReplaceTempView('icd10HccView')

best_icd_mapped_df=spark.read.load(f'{delta_path}/gold/best-icd-mapped')
best_icd_mapped_df.createOrReplaceTempView('bestIcdMappedView')
best_icd_mapped_pdf=best_icd_mapped_df.toPandas()

In [0]:
%sql
select * from icd10HccView
limit 10

path,final_chunk,entity,icd10_code,confidence,all_codes,resolutions,hcc_status,hcc_score,billable,icd_codes_names,icd_code_billable
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Mesothelioma,Oncological,C45,0.9986,"List(C45, C450, C459, C452, C457, C451, G731, C439, D165, C717, C649, C710, D352, C9000, C900)","List(mesothelioma [Mesothelioma], mesothelioma of pleura [Mesothelioma of pleura], mesothelioma, unspecified [Mesothelioma, unspecified], mesothelioma of pericardium [Mesothelioma of pericardium], mesothelioma of other sites [Mesothelioma of other sites], mesothelioma of mesentery [Mesothelioma of peritoneum], cancer, mesothelioma [Lambert-Eaton syndrome in neoplastic disease], amelanotic melanoma [Malignant melanoma of skin, unspecified], ameloblastoma of mandible [Benign neoplasm of lower jaw bone], glioma of brainstem [Malignant neoplasm of brain stem], nephroblastoma [Malignant neoplasm of unspecified kidney, except renal pelvis], glioblastoma multiforme, cerebrum [Malignant neoplasm of cerebrum, except lobes and ventricles], pituitary microadenoma [Benign neoplasm of pituitary gland], smoldering myeloma [Multiple myeloma not having achieved remission], smoldering myeloma [Multiple myeloma])","List(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0)","List(0, 9, 9, 9, 9, 9, 75, 12, 0, 10, 11, 10, 12, 9, 0)","List(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)",mesothelioma,0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Mesothelioma,Oncological,C45,0.9986,"List(C45, C450, C459, C452, C457, C451, G731, C439, D165, C717, C649, C710, D352, C9000, C900)","List(mesothelioma [Mesothelioma], mesothelioma of pleura [Mesothelioma of pleura], mesothelioma, unspecified [Mesothelioma, unspecified], mesothelioma of pericardium [Mesothelioma of pericardium], mesothelioma of other sites [Mesothelioma of other sites], mesothelioma of mesentery [Mesothelioma of peritoneum], cancer, mesothelioma [Lambert-Eaton syndrome in neoplastic disease], amelanotic melanoma [Malignant melanoma of skin, unspecified], ameloblastoma of mandible [Benign neoplasm of lower jaw bone], glioma of brainstem [Malignant neoplasm of brain stem], nephroblastoma [Malignant neoplasm of unspecified kidney, except renal pelvis], glioblastoma multiforme, cerebrum [Malignant neoplasm of cerebrum, except lobes and ventricles], pituitary microadenoma [Benign neoplasm of pituitary gland], smoldering myeloma [Multiple myeloma not having achieved remission], smoldering myeloma [Multiple myeloma])","List(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0)","List(0, 9, 9, 9, 9, 9, 75, 12, 0, 10, 11, 10, 12, 9, 0)","List(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)",mesothelioma,0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Mesothelioma,Oncological,C45,0.9986,"List(C45, C450, C459, C452, C457, C451, G731, C439, D165, C717, C649, C710, D352, C9000, C900)","List(mesothelioma [Mesothelioma], mesothelioma of pleura [Mesothelioma of pleura], mesothelioma, unspecified [Mesothelioma, unspecified], mesothelioma of pericardium [Mesothelioma of pericardium], mesothelioma of other sites [Mesothelioma of other sites], mesothelioma of mesentery [Mesothelioma of peritoneum], cancer, mesothelioma [Lambert-Eaton syndrome in neoplastic disease], amelanotic melanoma [Malignant melanoma of skin, unspecified], ameloblastoma of mandible [Benign neoplasm of lower jaw bone], glioma of brainstem [Malignant neoplasm of brain stem], nephroblastoma [Malignant neoplasm of unspecified kidney, except renal pelvis], glioblastoma multiforme, cerebrum [Malignant neoplasm of cerebrum, except lobes and ventricles], pituitary microadenoma [Benign neoplasm of pituitary gland], smoldering myeloma [Multiple myeloma not having achieved remission], smoldering myeloma [Multiple myeloma])","List(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0)","List(0, 9, 9, 9, 9, 9, 75, 12, 0, 10, 11, 10, 12, 9, 0)","List(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)",mesothelioma,0
dbfs:/FileStore/HLS/nlp/data/mt_on

In [0]:
%sql
select entity, count('*') from icd10HccView
group by 1
order by 2

entity,count(*)
Treatment,90
Oncological,441
Symptom,968


## 1.1. Get general information for staff management, reporting, & planning.

Let's take a look at the distribution of mapped codes

In [0]:
display(
  best_icd_mapped_df
  .select('onc_code_desc')
  .filter("onc_code_desc!='-'")
  .groupBy('onc_code_desc')
  .count()
  .orderBy('count')
)

onc_code_desc,count
Aplastic and other anemias and other bone marrow failure syndromes,3
Other disorders of the nervous system,3
In situ neoplasms,4
Metabolic disorders,6
Persons encountering health services for examinations,6
Malignant neoplasms of digestive organs,26
Malignant neoplasms of female genital organs,41
Melanoma and other malignant neoplasms of skin,51
"Malignant neoplasms of ill-defined, other secondary and unspecified sites",53
"Malignant neoplasms of lymphoid, hematopoietic and related tissue",85


we can also visualize the results as a countplot to see the number of each parent categories

In [0]:
import plotly.graph_objects as go

_ps=best_icd_mapped_pdf['onc_code_desc'].value_counts()
data=_ps[_ps.index!='-']

fig = go.Figure(go.Bar(
            x=data.values,
            y=data.index,
            orientation='h'))
fig.show()

## 1.2. Reimbursement-ready data with billable codes
In the previous notebook, using an icd10 oncology mapping dictionary, we created a dataset of coded conditions that are all billable. To assess the quality of the mapping, we can look at the distribution of 
the nearest billable codes

In [0]:
import plotly.express as px

_ps=best_icd_mapped_pdf['nearest_billable_code_pos'].value_counts()
data=_ps[_ps!='-']
data_pdf=pd.DataFrame({"count":data.values,"Index of Nearest Billable Codes":data.index})

fig = px.bar(data_pdf, x='Index of Nearest Billable Codes', y='count')
fig.show()

## 1.3. See which indications have the highest average risk factor
In our pipeline we used `sbiobertresolve_icd10cm_augmented_billable_hcc` as a sentence resolver, in which the model return HCC codes. We can look at the distribution risk factors for each entity.
Note that since each category has a different number of corresponding data points, to get a full picture of the distribution of risk factors for each condition, we use box plots.

In [0]:
import plotly.express as px

df = best_icd_mapped_pdf[best_icd_mapped_pdf.onc_code_desc!='-'].dropna()
fig = px.box(df, y="onc_code_desc", x="corresponding_hcc_score", hover_data=df.columns)
fig.show()

As we can see, some categories, even with fewer cases, have higher risk factor.

## 1.4. Analyze Oncological Entities
We can find the most frequent oncological entities.

In [0]:
onc_df = (
  icd10_hcc_df
  .filter("entity == 'Oncological'")
  .select("path","final_chunk","entity","icd10_code","icd_codes_names","icd_code_billable")
 )
onc_pdf=onc_df.toPandas()
onc_pdf.head(10)

Out[13]:

,path,final_chunk,entity,icd10_code,icd_codes_names,icd_code_billable
0,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_47.txt,cancer,Oncological,C801,cancer,1
1,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_47.txt,tumor cells,Oncological,D497,tumor of carotid body,1
2,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,Colon Cancer,Oncological,C189,cancer of the colon,1
3,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,colon cancer,Oncological,C189,cancer of the colon,1
4,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,T3c,Oncological,E119,t2dm,1
5,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,M0 colon cancer,Oncological,C9200,"leukemia, acute myelocytic, fab m0",1
6,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,tumor,Oncological,D446,carotid body tumor,1
7,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,colon cancer,Oncological,C189,cancer of the colon,1
8,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,colon cancers,Oncological,C189,"cancer of the colon, stage 3",1
9,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,colon cancer,Oncological,C189,cancer of the colon,1


In [0]:
import plotly.express as px

_ps=onc_pdf['icd_codes_names'].value_counts()
data=_ps[_ps.index!='-']
data_pdf=pd.DataFrame({"count":data.values,'icd code names':data.index})
data_pdf=data_pdf[data_pdf['count']>5]
fig = px.bar(data_pdf, y='icd code names', x='count',orientation='h')
fig.show()

### Report Counts by ICD10CM Code Names
Each bar shows count of reports contain the cancer entities.

In [0]:
display(
  onc_df.select('icd_codes_names','path')
  .dropDuplicates()
  .groupBy('icd_codes_names')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
)

icd_codes_names,count
carcinoma,10
cancer,10
carotid body tumor,8
breast cancer,6
breast cyst,6
ent symptoms,5
nephrotic syndrome w focal and segmental glomerular lesions,5
ekc,4
basal cell carcinoma of back,4
lymphoma,4


### Most common symptoms
 We can find the most common symptoms counting the unique symptoms in documents.

In [0]:
display(
  icd10_hcc_df
  .filter("lower(entity)='symptom'")
  .selectExpr('path','icd_codes_names as symptom')
  .dropDuplicates()
  .groupBy('symptom')
  .count()
  .orderBy(F.desc('count'))
  .limit(30)
)

symptom,count
edema,18
chest mass,17
distress,16
vesicular murmur,15
symptom occurs at night (finding),14
hepatosplenomegaly,13
pain,13
lymphadenopathy,12
amblyopic,11
nausea,11


### Extract most frequent oncological diseases and symptoms based on documents

Here, we will count the number documents for each symptom-disease pair. To do this, first we filter high confidence entities and then create a pivot table.

In [0]:
entity_symptom_df = (
  icd10_hcc_df
  .select('path','entity','icd_codes_names')
  .filter("lower(entity) in ('symptom','oncological') AND confidence > 0.30")
  .dropDuplicates()
)
display(entity_symptom_df)

path,entity,icd_codes_names
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,difficulty using urine bottle (finding)
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,syncope
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,congestion of nose
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_49.txt,Oncological,history of breast biopsy
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,loss of balance
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,worried
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,dysphagia
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_5.txt,Symptom,distress
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_48.txt,Symptom,blood in stool
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,muscle ache


In [0]:
condition_symptom_df = (
  entity_symptom_df.groupBy('path').pivot("entity").agg(F.collect_list("icd_codes_names"))
  .withColumnRenamed('Oncological','Condition')
  .withColumn('Conditions',F.explode('Condition'))
  .withColumn('Symptoms',F.explode('Symptom'))
  .drop('Condition','Symptom')
  .dropna()
  .fillna(0)
)
display(condition_symptom_df)

path,Conditions,Symptoms
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",dysphagia
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",swollen legs
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",blood in stool
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",dyspnea
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",difficulty in voiding
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",constipation
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",feeling tired
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",hemoptysis
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",odynophagia
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,"renal cell carcinoma, l kidney",cough


In [0]:
conditions_symptoms_count_df=condition_symptom_df.groupBy('Conditions').pivot("Symptoms").count().fillna(0)
conditions_symptoms_count_pdf=conditions_symptoms_count_df.toPandas()
conditions_symptoms_count_pdf.index=conditions_symptoms_count_pdf['Conditions']
conditions_symptoms_count_pdf=conditions_symptoms_count_pdf.drop('Conditions',axis=1)

In [0]:
selected_rows=conditions_symptoms_count_pdf.index[conditions_symptoms_count_pdf.sum(axis=1)>10]
selected_columns=conditions_symptoms_count_pdf.columns[conditions_symptoms_count_pdf.sum(axis=0)>10]

In [0]:
data_pdf=conditions_symptoms_count_pdf.loc[selected_rows,selected_columns]

Now let's visualize the heatmap of the co-occurence of conditions and symptoms. We can directly look at the counts of symptoms by condition

In [0]:
import plotly.express as px
def plot_heatmap(data,color='occurence'):
  fig = px.imshow(data,labels=dict(x="Condition", y="Symptom", color=color),y=list(data.index),x=list(data.columns))
  fig.update_layout(
    autosize=False,
    width=1100,
    height=1100,
  )
  fig.update_xaxes(side="top")
  return(fig)

In [0]:
fg=plot_heatmap(data_pdf)
fg.show()

As we see, this heatmap does not take the expected frequency of a given symptom into account. In order to reflect any correlation between the symptom in question and a given condition, we need to normalize the counts. 
To do so, we use `MinMaxScaler` to scale the values.

In [0]:
from sklearn.preprocessing import MinMaxScaler
normalized_data=MinMaxScaler().fit(data_pdf).transform(data_pdf)

In [0]:
norm_data_pdf=pd.DataFrame(normalized_data,index=data_pdf.index,columns=data_pdf.columns)
plot_heatmap(norm_data_pdf,'normalized occurence')

Out[25]:

As we can see, now the symptoms that were not appeared to be enriched show high correlation with corresponding conditions.

# 2. Get Drug codes from the notes

## Analyze drug usage patterns for inventory management and reporting

We are checking how many times any drug are encountered in the documents.

In [0]:
rxnorm_res_df=spark.read.load(f'{delta_path}/gold/rxnorm-res-cleaned')

In [0]:
display(
  rxnorm_res_df
  .filter('confidence > 0.8')
  .groupBy('drugs')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
)

drugs,count
carboplatin,14
heparin,11
prednisone,10
cyclosporine,8
aspirin,6
iron,5
doxorubicin,5
cyclophosphamide,5
fluconazole,4
epinephrine,4


# 3. Get Timeline Using RE Models

## Find the problems occured after treatments 

We are filtering the dataframe to select rows with following conditions to see problems occured after treatments.
* `relation =='AFTER'`
* `entity1=='TREATMENT'`
* `entity2=='PROBLEM'`

In [0]:
temporal_re_df=spark.read.load(f"{delta_path}/silver/temporal-re")

In [0]:
display(temporal_re_df)

path,relation,entity1,chunk1,entity2,chunk2,confidence
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,BEFORE,OCCURRENCE,Discharge,PROBLEM,Mesothelioma,0.99999833
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,BEFORE,OCCURRENCE,Discharge,DURATION,1,0.99978834
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,Mesothelioma,DURATION,1,0.9346253
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,Mesothelioma,PROBLEM,pleural effusion,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,Mesothelioma,PROBLEM,atrial fibrillation,0.99999607
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,Mesothelioma,PROBLEM,anemia,0.9996013
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,atrial fibrillation,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,anemia,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,ascites,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,esophageal reflux,0.94568694


In [0]:
display(
  temporal_re_df
  .where("relation == 'AFTER' AND entity1=='TREATMENT' AND entity2 == 'PROBLEM'")
  .filter('confidence > 0.8')
  .orderBy(F.desc('confidence'))
)

path,relation,entity1,chunk1,entity2,chunk2,confidence
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_21.txt,AFTER,TREATMENT,epinephrine,PROBLEM,a transverse incision,0.9999982
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_21.txt,AFTER,TREATMENT,Xylocaine,PROBLEM,a transverse incision,0.9999807
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_14.txt,AFTER,TREATMENT,this procedure,PROBLEM,complications,0.97644377
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_20.txt,AFTER,TREATMENT,intravenous heparin,PROBLEM,hereditary hypercoagulable state,0.9599375
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_37.txt,AFTER,TREATMENT,hemostatic,PROBLEM,a skin stab inferior,0.95746464


# 4. Analyze the Relations Between Body Parts and Procedures

In the extraction notebook, we created a relation extration model to identify relationships between body parts and problem entities by using pretrained **RelationExtractionModel** `re_bodypart_problem`. Now let's load the data and take a look at the relationship between bodypart and procedures. By filtering the dataframe to select rows satisfying `entity1 != entity2` we can see the relations between different entities and see the procedures applied to internal organs

In [0]:
bodypart_re_df=spark.read.load(f'{delta_path}/silver/bodypart-relationships')

In [0]:
display(
  bodypart_re_df
  .where('entity1!=entity2')
  .drop_duplicates()
  )

path,relation,entity1,chunk1,entity2,chunk2,confidence
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,IVC,Procedure,placement of a vena caval filter,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,1,Procedure,lumpectomy,Internal_organ_or_component,axillary node,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_35.txt,0,Internal_organ_or_component,prostate,Procedure,ultrasound-guided I-125 seed implantation,0.51350594
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,inferior vena cava,Procedure,placement of a vena caval filter,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,inferior vena cava,Procedure,mechanical and pharmacologic thrombolysis,0.9999087
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,1,Internal_organ_or_component,inferior vena cava,Procedure,balloon angioplasty,0.98969346
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_8.txt,0,Procedure,bone marrow biopsy,Internal_organ_or_component,cellular marrow,0.7271675
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_45.txt,0,Procedure,lymph node injection,Internal_organ_or_component,sentinel lymph node,0.9483816
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_33.txt,0,Internal_organ_or_component,nerve,Procedure,thyroidectomy,0.99999917
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,iliac vein,Procedure,placement of a vena caval filter,1.0


# 5. Get Procedure codes from notes

We will created dataset for procedure codes, using `jsl_ner_wip_greedy_clinical` NER mdodle and set NerConverter's WhiteList `['Procedure']` in order to get only drug entities. Let's take a look at this table:

In [0]:
cpt_df=spark.read.load(f'{delta_path}/silver/cpt')

In [0]:
display(cpt_df)

path,chunks,entity,cpt_code,confidence,all_codes,resolutions,cpt
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_32.txt,hysterectomy,Procedure,51925,0.3120,"List(51925, 58180, 58285, 58925, 59525, 58280, 19328, 58150, 59100, 44970, 19304, 44950, 19330, 27290, 58940, 19371, 31380, 38120, 44147, 63191, 31420, 62180, 45114, 45111, 58740)","List(Closure of vesicouterine fistula; with hysterectomy, Supracervical abdominal hysterectomy (subtotal hysterectomy), with or without removal of tube(s), with or without removal of ovary(s), Vaginal hysterectomy, radical (Schauta type operation), Ovarian cystectomy, unilateral or bilateral , Subtotal or total hysterectomy after cesarean delivery, Vaginal hysterectomy, with total or partial vaginectomy; with repair of enterocele, Removal of intact mammary implant, Total abdominal hysterectomy (corpus and cervix), with or without removal of tube(s), with or without removal of ovary(s), Hysterotomy, abdominal (e.g., for hydatidiform mole, abortion), Laparoscopy, surgical, appendectomy , Mastectomy, subcutaneous, Appendectomy, Removal of mammary implant material, Interpelviabdominal amputation (hindquarter amputation) , Oophorectomy, partial or total, unilateral or bilateral;, Periprosthetic capsulectomy, breast, Partial laryngectomy (hemilaryngectomy); anterovertical , Laparoscopy, surgical, splenectomy, Colectomy, partial; abdominal and transanal approach, Laminectomy with section of spinal accessory nerve, Epiglottidectomy , Ventriculocisternostomy (Torkildsen type operation), Proctectomy, partial, with anastomosis; abdominal and transsacral approach, Proctectomy; partial resection of rectum, transabdominal approach, Lysis of adhesions (salpingolysis, ovariolysis) )",Closure of vesicouterine fistula; with hysterectomy
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_32.txt,cataract surgery,Procedure,31592,0.0640,"List(31592, 50065, 61304, 61524, 61570, 50205, 61601, 61860, 61600, 61305, 61880, 62163, 43870, 50280, 61692, 61682, 61705, 61340, 61546, 47400, 44345, 49324, 61708, 44188, 33050)","List(Cricotracheal resection, Nephrolithotomy; secondary surgical operation for calculus , Craniectomy or craniotomy, exploratory; supratentorial, Craniectomy, infratentorial or posterior fossa; for excision or fenestration of cyst, Craniectomy or craniotomy; with excision of foreign body from brain, Renal biopsy; by surgical exposure of kidney , Resection or excision of neoplastic, vascular or infectious lesion of base of anterior cranial fossa; intradural, including dural repair, with or without graft, Craniectomy or craniotomy for implantation of neurostimulator electrodes, cerebral, cortical, Resection or excision of neoplastic, vascular or infectious lesion of base of anterior cranial fossa; extradural, Craniectomy or craniotomy, exploratory; infratentorial (posterior fossa), Revision or removal of intracranial neurostimulator electrodes, Neuroendoscopy, intracranial; with retrieval of foreign body, Closure of gastrostomy, surgical, Excision or unroofing of cyst(s) of kidney , Surgery of intracranial arteriovenous malformation; dural, complex, Surgery of intracranial arteriovenous malformation; supratentorial, complex, Surgery of aneurysm, vascular malformation or carotid-cavernous fistula; by intracranial and cervical occlusion of carotid artery, Subtemporal cranial decompression (pseudotumor cerebri, slit ventricle syndrome), Craniotomy for hypophysectomy or excision of pituitary tumor, intracranial approach, Hepaticotomy or hepaticostomy with exploration, drainage, or removal of calculus , Revision of colostomy; complicated (reconstruction in-depth) (separate procedure), Laparoscopy, surgical; with insertion of tunneled intraperitoneal catheter, Surgery of aneurysm, vascular malformation or carotid-cavernous fistula; by intracranial electrothrombosis, Laparoscopy, surgical, colostomy or skin level cecostomy, Resection of pericardial cyst or tumor)",Cricotracheal resection
dbfs:/FileStore/HLS/n

we can the see most common procedures being performed and count the number of each procedures and plot it.

In [0]:
#top 20
display(
  cpt_df
  .groupBy('cpt')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
)

cpt,count
Appendectomy,33
Thoracoscopy; with biopsy(ies) of pleura,30
Excision of presacral or sacrococcygeal tumor,19
Thoracotomy; with exploration,18
Resection of mediastinal tumor,10
Epiglottidectomy,9
Revision of peritoneal-venous shunt,8
"Closure of enterostomy, large or small intestine",8
Excision of perinephric cyst,6
Decortication and parietal pleurectomy,6


# 6. Get Assertion Status of Cancer Entities

Using the assertion status dataset we can find the number of family members of cancer patients with cancer or symptoms, and we can fruther check if the symptom is absent or present.

In [0]:
assertion_df=spark.read.load(f'{delta_path}/silver/assertion').drop_duplicates()

In [0]:
display(assertion_df)

path,chunk,entity,assertion
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_34.txt,edema,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,"focal motor, sensory or other neurological symptoms",Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,Newly diagnosed high-risk acute lymphoblastic leukemia,Oncological,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_40.txt,clubbing,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,narrowing,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,decreased appetite,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_7.txt,follicular non,Cancer,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,weight loss,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,chest pains,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_43.txt,shortness of breath,Symptom,absent


In [0]:
n_associated_with_someone_else = assertion_df.where("assertion=='associated_with_someone_else'").count()
print(f"Number of family members have cancer or symptoms: {n_associated_with_someone_else} ")

Number of family members have cancer or symptoms: 35

In [0]:
display(assertion_df)

path,chunk,entity,assertion
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_34.txt,edema,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,"focal motor, sensory or other neurological symptoms",Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,Newly diagnosed high-risk acute lymphoblastic leukemia,Oncological,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_40.txt,clubbing,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,narrowing,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,decreased appetite,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_7.txt,follicular non,Cancer,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,weight loss,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,chest pains,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_43.txt,shortness of breath,Symptom,absent


In [0]:
display(
  assertion_df
  .groupBy('assertion')
  .count()
)

assertion,count
present,571
hypothetical,13
conditional,12
possible,46
absent,500
associated_with_someone_else,35


In [0]:
assertion_symptom_df= (
  assertion_df
  .where("assertion in ('present', 'absent') AND entity=='Symptom'")
)
most_common_symptoms_df=(
  assertion_symptom_df
  .select('path','chunk')
  .groupBy('chunk')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
  )
display(most_common_symptoms_df)

chunk,count
edema,19
mass,15
murmurs,14
acute distress,12
hepatosplenomegaly,12
night sweats,12
pain,11
nausea,10
chills,10
lymphadenopathy,9


In [0]:
display(
  assertion_symptom_df
  .join(most_common_symptoms_df, on='chunk')
  .groupBy('chunk','assertion')
  .count()
  .orderBy(F.desc('count'))
  )

chunk,assertion,count
edema,absent,16
murmurs,absent,14
hepatosplenomegaly,absent,12
acute distress,absent,12
night sweats,absent,11
mass,present,10
chills,absent,10
pain,present,9
nausea,absent,9
vomiting,absent,9


## License
Copyright / License info of the notebook. Copyright [2021] the Notebook Authors.  The source in this notebook is provided subject to the [Apache 2.0 License](https://spdx.org/licenses/Apache-2.0.html).  All included or referenced third party libraries are subject to the licenses set forth below.

|Library Name|Library License|Library License URL|Library Source URL|
| :-: | :-:| :-: | :-:|
|Pandas |BSD 3-Clause License| https://github.com/pandas-dev/pandas/blob/master/LICENSE | https://github.com/pandas-dev/pandas|
|Numpy |BSD 3-Clause License| https://github.com/numpy/numpy/blob/main/LICENSE.txt | https://github.com/numpy/numpy|
|Apache Spark |Apache License 2.0| https://github.com/apache/spark/blob/master/LICENSE | https://github.com/apache/spark/tree/master/python/pyspark|
|Plotly|MIT License|https://github.com/plotly/plotly.py/blob/master/LICENSE.txt|https://github.com/plotly/plotly.py|
|Scikit Learn|BSD 3-Clause License|https://github.com/scikit-learn/scikit-learn/blob/main/COPYING|https://github.com/scikit-learn/scikit-learn|

## Disclaimers
Databricks Inc. (“Databricks”) does not dispense medical, diagnosis, or treatment advice. This Solution Accelerator (“tool”) is for informational purposes only and may not be used as a substitute for professional medical advice, treatment, or diagnosis. This tool may not be used within Databricks to process Protected Health Information (“PHI”) as defined in the Health Insurance Portability and Accountability Act of 1996, unless you have executed with Databricks a contract that allows for processing PHI, an accompanying Business Associate Agreement (BAA), and are running this notebook within a HIPAA Account.  Please note that if you run this notebook within Azure Databricks, your contract with Microsoft applies.